In [10]:
import psycopg2
import time
import threading
import random

In [11]:
username = 'ruslan'
password = '1111'
database = 'lab1DB'
host = 'localhost'
port = '5432'

In [12]:
def printdb(user_id):
    conn = psycopg2.connect(user=username, password=password, dbname=database, host=host, port=port)
    cursor = conn.cursor()

    cursor.execute("SELECT * FROM user_counter WHERE user_id = %s", (user_id,))
    row = cursor.fetchall()
    print(row)

    cursor.close()
    conn.close()

In [13]:
def create_db():
    conn = psycopg2.connect(user=username, password=password, dbname=database, host=host, port=port)
    cursor = conn.cursor()

    cursor.execute("""
        CREATE TABLE IF NOT EXISTS user_counter (
            USER_ID SERIAL PRIMARY KEY,
            Counter INTEGER,
            Version INTEGER
        );
    """)
    for i in range(1, 100001):
        cursor.execute("INSERT INTO user_counter (Counter, Version) VALUES (%s, %s)", (i, 0))
    conn.commit()
    cursor.close()
    conn.close()

# create_db()

In [14]:
def lost_update(user_id):
	
    conn = psycopg2.connect(user=username, password=password, dbname=database, host=host, port=port)
    cursor = conn.cursor()

    for _ in range(10000):
        cursor.execute("SELECT counter FROM user_counter WHERE user_id = %s", (user_id,))
        counter = cursor.fetchone()[0]
        counter += 1
        cursor.execute("UPDATE user_counter SET counter = %s WHERE user_id = %s", (counter, user_id,))
        conn.commit()

    cursor.close()
    conn.close()

In [15]:
def in_place_update(user_id):

    conn = psycopg2.connect(user=username, password=password, dbname=database, host=host, port=port)
    cursor = conn.cursor()

    for _ in range(10000):
        cursor.execute("UPDATE user_counter SET counter = counter + 1 where user_id = %s", (user_id,))
        conn.commit()

    cursor.close()
    conn.close()

In [16]:
def row_level_locking(user_id):
    
    conn = psycopg2.connect(user=username, password=password, dbname=database, host=host, port=port)
    cursor = conn.cursor()

    for _ in range(10000):
        cursor.execute(("SELECT counter FROM user_counter WHERE user_id = %s FOR UPDATE"), (user_id,))
        counter = cursor.fetchone()[0]
        counter += 1
        cursor.execute("UPDATE user_counter SET counter = %s WHERE user_id = %s", (counter, user_id,))
        conn.commit()

    cursor.close()
    conn.close()

In [17]:
def optimistic_concurrency_control(user_id):
    
    conn = psycopg2.connect(user=username, password=password, dbname=database, host=host, port=port)
    cursor = conn.cursor()

    for _ in range(10000):
        while True:
            cursor.execute("SELECT counter, version FROM user_counter WHERE user_id = %s", (user_id,))
            counter, version = cursor.fetchone()
            counter = counter + 1
            cursor.execute("UPDATE user_counter SET counter = %s, version = %s WHERE user_id = %s and version = %s", (counter, version + 1, user_id, version))
            conn.commit()
            count = cursor.rowcount
            if count > 0:
                break

    cursor.close()
    conn.close()

In [18]:
def execution_time(name_operation, target, user_id=1):

    threads = []
    start_time = time.time()
    for _ in range(10):
        thread = threading.Thread(target=target, args=(user_id,))
        threads.append(thread)
        thread.start()

    for t in threads:
        t.join()

    print(f"{name_operation}, user_id = {user_id}, time = {time.time() - start_time}s")

    conn = psycopg2.connect(user=username, password=password, dbname=database, host=host, port=port)
    cursor = conn.cursor()

    cursor.execute("UPDATE user_counter SET counter = %s, version = 0 where user_id = %s", (user_id, user_id,))
    conn.commit()

    cursor.close()
    conn.close()

execution_time("lost_update", lost_update)
execution_time("in_place_update", in_place_update)
execution_time("row_level_locking", row_level_locking)
execution_time("optimistic_concurrency_control", optimistic_concurrency_control)
print()

random_number = random.randint(1, 100000)
execution_time("lost_update", lost_update, random_number)
execution_time("in_place_update", in_place_update, random_number)
execution_time("row_level_locking", row_level_locking, random_number)
execution_time("optimistic_concurrency_control", optimistic_concurrency_control, random_number)

lost_update, user_id = 1, time = 41.334161043167114s
in_place_update, user_id = 1, time = 42.94075059890747s
row_level_locking, user_id = 1, time = 112.79331731796265s
optimistic_concurrency_control, user_id = 1, time = 189.1228792667389s

lost_update, user_id = 22304, time = 41.11386489868164s
in_place_update, user_id = 22304, time = 43.847917556762695s
row_level_locking, user_id = 22304, time = 107.99418568611145s
optimistic_concurrency_control, user_id = 22304, time = 181.58032131195068s


In [19]:
# lost_update, user_id = 1, time = 39.00321912765503s
# in_place_update, user_id = 1, time = 43.827351093292236s
# row_level_locking, user_id = 1, time = 108.34298801422119s
# optimistic_concurrency_control, user_id = 1, time = 193.67018604278564s

# lost_update, user_id = 36162, time = 37.0974497795105s
# in_place_update, user_id = 36162, time = 46.54637026786804s
# row_level_locking, user_id = 36162, time = 108.20453691482544s
# optimistic_concurrency_control, user_id = 36162, time = 197.93598079681396s